In [1]:
class Int:
    def __init__(self, min_value = None, max_value = None):
        self.min_value = min_value
        self.max_value = max_value

    def __set_name__(self, owner_class, property_name):
        self.property_name = property_name

    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError(f"{self.property_name} must be an integer.")

        if self.min_value is not None and value < self.min_value:
            raise ValueError(f"{self.property_name} must be greater than {self.min_value}")

        if self.max_value is not None and value > self.max_value:
            raise ValueError(f"{self.property_name} must be less than {self.max_value}")

        instance.__dict__[self.property_name] = value

    def __get__(self, instance, owner_class):
        if instance is None:
            return self

        return instance.__dict__.get(self.property_name)


In [2]:
class Point2D:
    x = Int(min_value=0, max_value=800)
    y = Int(min_value=0, max_value=600)

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f"<{type(self).__name__} x={self.x} y={self.y}> @ {hex(id(self))}"

    def __str__(self):
        return f"({self.x}, {self.y})"


In [3]:
p = Point2D(5, 25)
str(p), repr(p)

('(5, 25)', '<Point2D x=5 y=25> @ 0x105eeb500')

In [4]:
try:
    Point2D("a", "b")
except TypeError as e:
    print(e)

x must be an integer.


In [5]:
try:
    Point2D(-1, 20)
except ValueError as e:
    print(e)

x must be greater than 0


In [6]:
try:
    Point2D(1000, 500)
except ValueError as e:
    print(e)

x must be less than 800


In [7]:
import collections

In [8]:
isinstance([1, 2, 3], collections.abc.Sequence)

True

In [9]:
isinstance({1, 2, 3}, collections.abc.Sequence)

False

In [10]:
isinstance([1, 2, 3], collections.abc.MutableSequence)

True

In [11]:
isinstance((1, 2, 3), collections.abc.MutableSequence)

False

In [12]:
class Point2DSequence:
    def __init__(self, min_length = None, max_length = None):
        self.min_length = min_length
        self.max_length = max_length

    def __set_name__(self, owner_class, name):
        self.name = name

    def __get__(self, instance, owner_class):
        if instance is None:
            return self

        instance.__dict__.setdefault(self.name, [])
        return instance.__dict__.get(self.name)

    def __set__(self, instance, value):        
        if not isinstance(value, collections.abc.Sequence):
            raise TypeErrror(f"{self.name} must be an a Sequence type.")

        if self.min_length and len(value) < self.min_length:
            raise ValueError(f"{self.name} length must be greater or equal to {self.min_length}")

        if self.max_length and len(value) > self.max_length:
            raise ValueError(f"{self.name} length must be less or equal to {self.max_length}")

        if not all(isinstance(item, Point2D) for item in value):
            raise ValueError(f"{self.name} allows only Point2D to be used")

        instance.__dict__[self.name] = list(value)


In [13]:
class Polygon:
    vertices = Point2DSequence(min_length=3)

    def __init__(self, *vertices):
        self.vertices = vertices


In [14]:
try:
    Polygon()
except ValueError as e:
    print(e)

vertices length must be greater or equal to 3


In [15]:
try:
    Polygon(1, 2, 3, 4)
except ValueError as e:
    print(e)

vertices allows only Point2D to be used


In [16]:
p1 = Point2D(0, 0)
p2 = Point2D(0, 3)
p3 = Point2D(4, 0)

triangle = Polygon(p1, p2, p3)

In [17]:
triangle.__dict__

{'vertices': [<Point2D x=0 y=0> @ 0x105f46ba0,
  <Point2D x=0 y=3> @ 0x105f46b70,
  <Point2D x=4 y=0> @ 0x105f46bd0]}

In [18]:
class Polygon:
    vertices = Point2DSequence(min_length=3, max_length=4)

    def __init__(self, *vertices):
        self.vertices = vertices

    def append(self, point_2d):
        if not isinstance(point_2d, Point2D):
            raise TypeError("Only Point2D instances are allowed")

        max_allowed_length = type(self).vertices.max_length
        if max_allowed_length and len(self.vertices) >= max_allowed_length:
            raise ValueError("Max allowed length exceeded")

        self.vertices.append(point_2d)


In [19]:
p1 = Point2D(0, 0)
p2 = Point2D(0, 3)
p3 = Point2D(4, 0)

triangle = Polygon(p1, p2, p3)

In [20]:
triangle.append(p2)

In [21]:
triangle.__dict__

{'vertices': [<Point2D x=0 y=0> @ 0x105f47dd0,
  <Point2D x=0 y=3> @ 0x105f47a10,
  <Point2D x=4 y=0> @ 0x105f47da0,
  <Point2D x=0 y=3> @ 0x105f47a10]}

In [22]:
try:
    triangle.append(p2)
except ValueError as e:
    print(e)

Max allowed length exceeded
